### Steps to be followed

* Making the connection between RDS and SQL server
* Installing and importing the necessary libraries
* Making the connection between RDS and sagemaker
* Fetching the data
* Some insights on the data

#### Making a connection between RDS and SQL server

Create a database in RDS and then make a connection with the SQL server using the endpoints.

In [13]:
pip install pymssql

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


pymssql library is not present in the sagemaker by default so install this library to make a connection between sagemaker and RDS.

In [1]:
#Importing the necessary libraries

import pymssql
import numpy as np
import pandas as pd

#### Making the connection between RDS and sagemaker

In [2]:
connection={
    'host': 'database-1.cnvhfuv3ncfl.us-east-1.rds.amazonaws.com',      #server name that is endpoint of database in rds
    'username': 'admin',                                                #provided while creating the database
    'password': '750vip12',
    'db': 'aws-orders-db'                                               #Name of the database
}

In [3]:
#making a connection

con=pymssql.connect(connection['host'],connection['username'],connection['password'],connection['db'])
cursor=con.cursor()

### Fetching the data

In [4]:
#Importing the categories table from the orders database

select_statement1="""select * from categories$"""
cursor.execute(select_statement1)
columns = cursor.description          #Fetching complete raw data
result1 = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]   #Get the index column names along with the data
categories=pd.DataFrame(result1)      #Converting that data into a dataframe
categories.head()

,CategoryID,CategoryName,Description
0,1.0,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2.0,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3.0,Confections,"Desserts, candies, and sweet breads"
3,4.0,Dairy Products,Cheeses
4,5.0,Grains/Cereals,"Breads, crackers, pasta, and cereal"


Imported the **Categories** table from orders database which have three features including categoryid, category name and description of the categories.

In [5]:
#Importing the customers table from the orders database

select_statement2="""select * from customers$"""
cursor.execute(select_statement2)
columns = cursor.description           #Fetching complete raw data
result2 = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]    #Get the index column names along with the data
customers=pd.DataFrame(result2)        #Converting that data into a dataframe
customers.head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1.0,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209.0,Germany
1,2.0,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021.0,Mexico
2,3.0,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023.0,Mexico
3,4.0,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,NaN,UK
4,5.0,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,NaN,Sweden


Imported the **Customers** table from orders database which have seven features regarding the basic information about the customer.

In [6]:
#Importing the orderdetails table from the orders database

select_statement3="""select * from orderdetails$"""
cursor.execute(select_statement3)
columns = cursor.description               #Fetching complete raw data
result3 = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]    #Get the index column names along with the data
orderdetails=pd.DataFrame(result3)         #Converting that data into a dataframe
orderdetails.head()

,OrderDetailID,OrderID,ProductID,Quantity
0,1.0,10248.0,11.0,12.0
1,2.0,10248.0,42.0,10.0
2,3.0,10248.0,72.0,5.0
3,4.0,10249.0,14.0,9.0
4,5.0,10249.0,51.0,40.0


Imported the **Orderdetails** table from orders database which have four features including Id's and quantity of the products sold.

In [7]:
#Importing the orders table from the orders database

select_statement4="""select * from orders$"""
cursor.execute(select_statement4)
columns = cursor.description               #Fetching complete raw data
result4 = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
orders=pd.DataFrame(result4)               #Converting that data into a dataframe
orders.tail()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
191,10439.0,51.0,6.0,1997-02-07,3.0
192,10440.0,71.0,4.0,1997-02-10,2.0
193,10441.0,55.0,3.0,1997-02-10,2.0
194,10442.0,20.0,3.0,1997-02-11,2.0
195,10443.0,66.0,8.0,1997-02-12,1.0


Imported the **Orders** table from orders database which have five features including some important Id's.

### Some insights

#### How many distinct customers are there?

In [8]:
#Count of the customer id from customers table

customers['CustomerID'].count()

91

#### How many customers are there by each country?

In [9]:
#count of the customer id on the basis of the country name

customer_country =  customers.groupby('Country',as_index=False).agg({'CustomerID':'count'})
customer_country.sort_values('CustomerID',ascending=False,ignore_index= True)

,Country,CustomerID
0,USA,13
1,Germany,11
2,France,11
3,Brazil,9
4,UK,7
5,Spain,5
6,Mexico,5
7,Venezuela,4
8,Argentina,3
9,Canada,3


Highest number of customers are from USA and lowest number of customers are from Ireland, Norway and Poland.

#### How my products each customer has purchased?

* Merge the orderdetails and orders table on the common key order id.
* count of product id on the basis of the customer id.

In [13]:
#Merge the orderdetails and orders table on the common key order id.

orders_details=pd.merge(orderdetails,orders, on='OrderID', how='left')
orders_details.head()

,OrderDetailID,OrderID,ProductID,Quantity,CustomerID,EmployeeID,OrderDate,ShipperID
0,1.0,10248.0,11.0,12.0,90.0,5.0,1996-07-04,3.0
1,2.0,10248.0,42.0,10.0,90.0,5.0,1996-07-04,3.0
2,3.0,10248.0,72.0,5.0,90.0,5.0,1996-07-04,3.0
3,4.0,10249.0,14.0,9.0,81.0,6.0,1996-07-05,1.0
4,5.0,10249.0,51.0,40.0,81.0,6.0,1996-07-05,1.0


In [11]:
#count of product id on the basis of the customer id.

products_customer = orders_details.groupby('CustomerID',as_index=False).agg({'ProductID':'count'})
products_customer.sort_values('ProductID',ascending=False,ignore_index= True)

,CustomerID,ProductID
0,20.0,35
1,65.0,22
2,37.0,21
3,63.0,20
4,71.0,16
...,...,...
69,11.0,2
70,91.0,2
71,3.0,1
72,47.0,1


The top customer with highest purchase is with customer id 20.

#### How many products each employee has sold? Which employee has sold maximum?

In [12]:
#From the merged table orders_details, count of product id on the basis of employee id

products_employees = orders_details.groupby('EmployeeID',as_index=False).agg({'ProductID':'count'})
products_employees.sort_values('ProductID',ascending=False,ignore_index= True)

,EmployeeID,ProductID
0,4.0,123
1,1.0,78
2,3.0,74
3,8.0,68
4,2.0,49
5,6.0,46
6,7.0,34
7,5.0,27
8,9.0,19


The employee with employee id 4 has sold maximum number of products.